In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import scipy
from scipy import optimize
from scipy.optimize import curve_fit

In [ ]:
#Preparing the file
with open(r"C:\Users\eneto\Documents\GitHub\ASTR19-TidalReport\ASTR19_S22_group_project_data.txt") as file:
    data = file.read()
    
def parse_tide_data(data):
    rows = []
    for line in data.split('\n'):
        parts = line.split()
        if len(parts) == 3:
            day, time, height = parts
            rows.append({'Day': int(day), 'Time': time, 'Tide Height (feet)': float(height)})
            
    return pd.DataFrame(rows)
df = parse_tide_data(data)

# converting time and hours to a decimal point
def time_to_hours(t):
    hours, minutes = map(int, t.split(':'))
    return hours + minutes / 60
df['Time (Hours)'] = df['Time'].apply(time_to_hours)
# creating a standardized timestamp that can be plotted on the x-axis
df['Timepoint'] = df['Day'] + df['Time (Hours)'] / 24

In [ ]:
#We experimented with a 1-sine model, a 2-sine model, and a 3-sine model, and the 2-sine model gave up the best results.
def oscillatory_function(x, a1, b1, c1, d1, a2, b2, c2, d2):
    return a1*np.sin(b1*x+c1)+d1 + a2*np.sin(b2*x+c2)+d2

parameters = [5, 0.5, 0, 0,   5, 2*np.pi, 1, 0]

#Now fitting the curve to the data...with mixed success, unfortunately.
params, params_cov = optimize.curve_fit(oscillatory_function, df['Timepoint'], df['Tide Height (feet)'],
                      sigma=np.full(len(df), 0.25),p0=parameters)

fit = oscillatory_function(df['Timepoint'], *params)

In [ ]:
y_err=np.full(82,.25)
plt.figure(figsize=(20, 5))
plt.plot(df['Timepoint'], fit, label='Best Fit', color='red') 
plt.errorbar(df['Timepoint'], df['Tide Height (feet)'],yerr=y_err,fmt='o', label='Data', color='blue')

plt.xlabel('Timepoint (Day + Time in Hours)')
plt.ylabel('Tide Height (feet)')
plt.title('Tide Height vs. Time')
plt.legend()
plt.savefig('data_and_fit.png',bbox_inches='tight',dpi=700)
plt.show()

I think that the data needed a third sine function, but no matter what parameters I used, I couldn't get it to behave. 

In [ ]:
sub = df['Tide Height (feet)']-fit
plt.figure(figsize=(20,5))
plt.scatter(df['Timepoint'],sub,label='Residuals',color="green")
plt.xlabel('Timepoint (Day + Time in Hours)')
plt.ylabel('Tide Height (feet)')
plt.yticks(ticks=(np.arange(-2,3,1)))
plt.title('Tide Height vs. Time')
plt.legend()
plt.savefig('residuals.png',bbox_inches='tight',dpi=700)
plt.show()

In [ ]:
plt.figure(figsize=(20,5))
width=.1
histmin=np.floor(min(sub))
histmax=np.ceil(max(sub))+width
bins=np.arange(histmin,histmax,width)
plt.hist(sub,bins=bins,alpha=.5,edgecolor='black')
plt.ylabel('# of Residuals')
plt.xlabel('Deviation from fit in ft')
plt.title('Residual deviation from the fit (w/o tsunami)')
plt.xticks(ticks=(np.arange(-2,2.4,.1)))
plt.savefig('hist_no_tsunami.png',bbox_inches='tight',dpi=700)
plt.show()
print(f'The standard deviation of the residuals is {np.std(sub)} ft, which is greater than the assumed experimental error, 0.25 ft.')

In [ ]:
df.iat[26,2] = 7.33
sub = df['Tide Height (feet)']-fit
stddev = np.std(sub)
plt.figure(figsize=(20,5))
width=.1
histmin=np.floor(min(sub))
histmax=np.ceil(max(sub))+width
bins=np.arange(histmin,histmax,width)
plt.hist(sub,bins=bins,alpha=.5,edgecolor='black')
plt.ylabel('# of Residuals')
plt.xlabel('Deviation from fit in ft')
plt.title('Residual deviation from the fit (w/tsunami)')
plt.xticks(ticks=(np.arange(-2,2.4,.1)))
plt.savefig('hist_tsunami.png',bbox_inches='tight',dpi=700)
plt.show()
print(f'The standard deviation of the residuals is {stddev} ft, which is greater than the assumed experimental error, 0.25 ft.')
print(f"The significance of the outlier on the 14th day is {(2-2.48)/0.92} sigma.")